<a href="https://colab.research.google.com/github/andydaehn/Kaggle/blob/main/California_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Packages

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt

import lightgbm as lgbm
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_california_housing

## EDA

In [ ]:
#DATAPATH = Path('../input/playground-series-s3e1')

train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission.csv')